# <mark>Processa pipeline models - 2_process_pipe_unificado_V1</mark>

Atual notebook com as funçoes para processamento de PDF Pesquisavel e Raster

### Import

In [1]:
import os
import csv
import sys
import json
import shutil


import platform
from io import StringIO
from pathlib import Path
from unidecode import unidecode
from unicodedata import normalize



from pdf2image import convert_from_path
import matplotlib.pyplot as plt


import re
import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw

import pytesseract

from fuzzywuzzy import fuzz
from unidecode import unidecode
from unicodedata import normalize
import PyPDF2

import pandas as pd



from datetime import datetime, timezone, timedelta

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar


import pandas as pd
import locale
from datetime import datetime, timezone, timedelta
import time, copy


from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr


from pytz import timezone
from urllib import response



import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron

### Config

In [11]:
i_test = 17

### PRESTAR ATENCAO
#model = 'mage'
#model = 'mesquita'
model = 'pedro_aldeia'


# Nome Batch
batch_name = "Batch_" + str(i_test)

root_doc_analise = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"

######### PATHS
#1. path formado para busca de pdfs recursiva
root_doc_analise = os.path.join(root_doc_analise, batch_name)

# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# Nome do arquivo json
nome_formado_json = batch_name +".json"


# 1. path para documentos PDF (omelhor se estiverem dentro de um unico diretorio)
root_pdf_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 2. path para documentos PDF que podem estar aguardando para serem processados
root_pdf_aguardando_path = "pipeline_extracao_documentos/3_tratamento_excecoes/pdf_aguardando_processar"

documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 3. path para documentos PDF externos para serem processados
root_external_pdf_path = "content_from_pdftool/data/data_pdf/NF_para_processamento/NFRJ_PDF_para _ocr"
# 4. path para documentos PDF PESQUISAVEIS externos para serem processados
root_external_pdf_pesquisavel_path = "content_from_pdftool/data/data_pdf/NF_processadas/NFRJ/fwdnotasfiscaisemitidaslmpadalegal"

# 5. path para imagem padrao
image_resized_path = 'pipeline_extracao_documentos/6_geral_administacao/images/processadas'

# 6. path para log
log_path = 'pipeline_extracao_documentos/6_geral_administacao/logs'

# 8. path para NFs processadas
nf_processada_path = "pipeline_extracao_documentos/5_documentos_processados"

#### paths de objetos para criacao/gestao (dicionarios/datasets)
cnae_dict_path = "pipeline_extracao_documentos/6_geral_administacao/datasets/MAGE_CNAE_X_ITEM_SERVICO_V1.xlsx"

# VERIFICAR
tgt_imagens = "pipeline_extracao_documentos/6_geral_administacao/images"

# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")


#### E-mail

documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments'

# 3. Path para documentos atachados:
documentos_extracao_path = 'pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento'




'pt_BR.utf8'

In [5]:
print(f'\ni_test: {i_test} | model: {model} | batch_name: {batch_name} | root_doc_analise: {root_doc_analise}\n')


i_test: 13 | model: mage | batch_name: Batch_13 | root_doc_analise: pipeline_extracao_documentos/2_documentos_para_extracao/Batch_13



In [3]:
# 1. Tratando nome de carga do df_processamento
dataset_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"

df_processamento_file = "df_processamento_"

df_processamento_file_read = df_processamento_file + str(i_test - 1) + ".xlsx"

# 2. Tratando nome de carga do df_extracao_files_Batch
df_extracao_files_Batch_file = "df_extracao_files_Batch_"

df_extracao_files_Batch_file_read = df_extracao_files_Batch_file + str(i_test - 1) + ".xlsx"


df_processamento_file_read_path = os.path.join(dataset_path, df_processamento_file_read)

df_processamento_file_read_path

df_extracao_files_Batch_file_read_path = os.path.join(dataset_path, df_extracao_files_Batch_file_read)

df_extracao_files_Batch_file_read_path

df_processamento = pd.read_excel(df_processamento_file_read_path) 

df_processamento

df_extracao_files = pd.read_excel(df_extracao_files_Batch_file_read_path) 

df_extracao_files

,index,Batch,diretorio_origem,nome_arquivo_origem,nome_arquivo_destino,data_processamento,tipo_pdf,qut_paginas
0,2,Batch_12,notas_mage_31082023,NF 689- PMMacae- 8ª Med CO-22-22 TB Contrato 0...,nf_689__pmmacae__8a_med_co_22_22_tb_contrato_0...,01/09/2023 14:22:44,False,1
1,2,Batch_12,notas_mage_31082023,nfse 4414.pdf,nfse_4414.pdf,01/09/2023 14:22:44,True,1
2,2,Batch_12,notas_mage_31082023,nfse 4409.pdf,nfse_4409.pdf,01/09/2023 14:22:44,True,1
3,2,Batch_12,notas_mage_31082023,NF 688 - ConsoR Zadar- Engetecnica (ENZA) (1).pdf,nf_688___consor_zadar__engetecnica_enza_1.pdf,01/09/2023 14:22:44,False,1
4,2,Batch_12,notas_mage_31082023,12225 COFIM -.pdf,12225_cofim__.pdf,01/09/2023 14:22:44,True,1
...,...,...,...,...,...,...,...,...
78,4,Batch_16,159871,2023 -8.pdf,2023__8.pdf,01/09/2023 15:28:23,True,1
79,5,Batch_16,160014,31-07.pdf,31_07.pdf,01/09/2023 15:28:23,True,1
80,5,Batch_16,160014,ACFrOgBLgYewSPQAweUd3QJkpDqN5Kp2dFIyNq7d6wJCRY...,acfrogblgyewspqaweud3qjkpdqn5kp2dfiynq7d6wjcry...,01/09/2023 15:28:23,True,1
81,6,Batch_16,126623,41C46D8F-73AB-4906-A4C6-C7DC92C05828.PDF,41c46d8f_73ab_4906_a4c6_c7dc92c05828.pdf,01/09/2023 15:28:23,True,1


### trata dicionarios

In [12]:
# 9. path para models
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v9.xlsx"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"


#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']




mage_cnae_x_item_servico_df = pd.read_excel(cnae_dict_path)

# Creating a dictionary for CNAE codes and descriptions
cnae_dict = dict(zip(mage_cnae_x_item_servico_df['cnae'], mage_cnae_x_item_servico_df['descricao_cnae']))
item_servico_dict = dict(zip(mage_cnae_x_item_servico_df['item_servico'], mage_cnae_x_item_servico_df['descricao_item_servico']))




In [5]:
# 2. Leitura do arquivo CSV e criação do dicionário modelos
def create_model_dictionary(model_dict_path):
    model_dictionary = {}
    with open(model_dict_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            prefeitura_name = row['prefeitura']
            model_name = row['model']

            if prefeitura_name not in model_dictionary:
                model_dictionary[prefeitura_name] = model_name
            
            #model_dictionary[prefeitura_name].append(model_name)
    
    return model_dictionary


# 3. Cria o dict de modelos
model_dict = create_model_dictionary(model_dict_path) 

NameError: name 'model_dict_path' is not defined

In [6]:
if model_dict[name_prefeitura]:
    modelo = model_dict[name_prefeitura]
    model = model_dict[name_prefeitura]
    message_log = f"Nome de prefeitura: {name_prefeitura} encontrada para: {doc2convert} modelo a ser utilizado: {modelo}"
    print(message_log)
    #logging.info(message_log) 
else:
    modelo = model_padrao
    message_log = f'\nmodelo nao encontrado para prefeitura {name_prefeitura}, sera utilizado o modelo {modelo}'   
    print(message_log)
    #logging.info(message_log)   

NameError: name 'model_dict' is not defined

### funcoes de arquivo - IMPORTANTE

In [8]:
## Modelo para criar varias paginas do PDF

import fitz  # PyMuPDF

# Caminho para o PDF que você quer dividir
input_pdf_path = "seuarquivo.pdf"

# Abre o PDF
pdf = fitz.open(input_pdf_path)

# Número total de páginas no PDF
total_pages = len(pdf)

# Nome base para os arquivos de saída
base_name = input_pdf_path.split('.')[0]  # Remove a extensão do arquivo

# Loop para criar um novo PDF para cada página
for page_num in range(total_pages):
    # Cria um novo objeto PDF
    new_pdf = fitz.open()
    # Adiciona a página atual ao novo PDF
    new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
    # Nome do novo arquivo PDF
    new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
    # Salva o novo PDF
    new_pdf.save(new_pdf_name)
    # Fecha o novo PDF
    new_pdf.close()

# Fecha o PDF original
pdf.close()


FileNotFoundError: no such file: 'seuarquivo.pdf'

In [ ]:
# move NF processadas ok
def move_pdf_processed_ok(document_path, nf_processada_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(nf_processada_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message

In [ ]:
def move_pdf_processed_ok(document_path):
    
    source_path = document_path
    destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')
    shutil.move(source_path, destination_path)
    

In [ ]:
# mover PDF para diretorio de processados
source_path = document_path
destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')
shutil.move(source_path, destination_path)

In [ ]:
# ***DELETAR**** o arquivo de imagem gerado
image_path_to_delete = name_image_2work
os.remove(image_path_to_delete)

In [ ]:
# voltar (mover) PDF para diretorio para ser processado
source_path = destination_path

destination_path = document_path

shutil.move(source_path, destination_path)

In [ ]:
# Processo para iterar estrutura de arquivos, diretorios
for roots, directories, documents in os.walk(image_resized_path):
    print(f'\n {documents} {name_image_2work}\n\n')
    #if documents == :
        

In [ ]:
# 7. Buscar o primeiro documento do diretorio EXTERNO para processar
source_directory = root_external_pdf_path
destination_directory = root_pdf_path

# Verifique se o diretório de destino está vazio
if os.listdir(destination_directory):
    raise Exception("O diretório de destino não está vazio!")

for roots, directories, documents in os.walk(source_directory):
    # Filtre os documentos para incluir apenas aqueles com extensão .pdf
    pdf_files = [doc for doc in documents if doc.lower().endswith('.pdf')]
    
    if pdf_files: # Verifique se há algum arquivo PDF no diretório
        first_pdf_file = pdf_files[0] # Obtenha o primeiro arquivo PDF
        source_path = os.path.join(roots, first_pdf_file) # Construa o caminho completo para o primeiro arquivo PDF
        print(f"Found the first PDF file: {source_path}")
        
        # Mova o arquivo para o diretório de destino
        shutil.move(source_path, destination_directory)
        
        break # Saia do loop após encontrar o primeiro arquivo PDF

### 4. Funcoes necessarias 

In [25]:
# 1. converte nome do arquivo
def conv_filename(title):
    
    # Divide o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remove acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substiti espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remove quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converte para minúsculas
    filename = filename.lower()

    # Adiciona a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 

def confirma_pdf_pequisavel(file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical
    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if text:
       page_number = 0
       pdf_realmente_pequisavel = True
       #print(page_number)
    else:
       page_number = 1
       pdf_realmente_pequisavel = False
       #print(page_number)
    
    try:
        page = pdf_document[page_number]
        x0 = 0
        y0 = 0
        x1 = 600
        y1 = 110

        text = page.get_text("text", clip=(x0, y0, x1, y1))
        nf_data_cabecalho = Extc.extract_fields_cabecalho(text)
        
        if nf_data_cabecalho:
            pdf_realmente_pequisavel = True
        else:
            pdf_realmente_pequisavel = False
            
      
        return pdf_realmente_pequisavel, page_number
    except Exception as e:
        print(f"Erro ao abrir pagina do PDF: {e}")
        return pdf_realmente_pequisavel, page_number
    pdf_document.close()   

def confirma_pdf_pequisavelx(file, file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical
    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if text:
       page_number = 0
       print(page_number)
    else:
       page_number = 1
       print(page_number)
    
    try:
        page = pdf_document[page_number]
        x0 = 0
        y0 = 0
        x1 = 600
        y1 = 110

        text = page.get_text("text", clip=(x0, y0, x1, y1))
        nf_data_cabecalho = Extc.extract_fields_cabecalho(text)
        
        if nf_data_cabecalho:
            pdf_realmente_pequisavel = True
        else:
            pdf_realmente_pequisavel = False
            
        pdf_document.close()
        
        return pdf_realmente_pequisavel
    except Exception as e:
        print(f"Erro ao abrir pagina do PDF: {e}")
        return pdf_realmente_pequisavel


# 2. Pesquisa prefeitura no documento
def pequisaModel(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1) 
            return  nome_prefeitura  
    


# 3. move NF processadas ok
def move_raster_pdf(document_path, raster_pdf_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(raster_pdf_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message

# 5. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable_analise(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        pages = pdf_document.page_count
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf, pages
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

### 5. Lendo o email e anexos

<mark> <b>1.</b> Lendo email e anexos </mark>

In [13]:
# Defina a localização para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

for root, dirs, files in os.walk(msg_dir_path):
    #print(f'{root}  | {dirs} | {document} | {files}\n')
    
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.msg'):
            email_message = file
            email_path = os.path.join(root, file)
            if email_message:
                print(f'email: {email_message}, email_path: {email_path}')
            else:
                print("Nao ha email")
                break
try:            
    msg = extract_msg.Message(email_path)
    msg_raw_sender = msg.sender
    parts = msg_raw_sender.rsplit('<', 1)
    msg_email_address = parts[1].strip('<>')
    msg_sender = parts[0].strip(' ')
    msg_subject = msg.subject
    msg_body = msg.body
    # String original
    original_date_str = msg.date
    date_email = cron.convert_email_date(original_date_str)
        
    # Caminho da pasta onde você quer salvar os anexos
    pasta_destino = msg_attachment_zip

    # Verifica se a pasta existe; se não, cria ela
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    with open(email_path) as msg_file:
        msg = Message(msg_file)
    
    total_attch = len(msg.attachments)
    
    print(total_attch)
    
except Exception as e:
    print(f"Erro ao ler email: {e}")
    

arquivos_zip = []
arquivos = []
i = 0
# Loop para salvar cada anexo
for i in range(total_attch):
    attachment = msg.attachments[i]
    caminho_completo_anexo = os.path.join(pasta_destino, attachment.filename)
    if file.lower().endswith('.zip') or file.lower().endswith('.rar') or file.lower().endswith('.7z'):
        arquivos_zip.append(attachment.filename)
    else:
        arquivos.append(attachment.filename)
      
    print(caminho_completo_anexo)
    with attachment.open() as attachment_fp, open(caminho_completo_anexo, 'wb') as output_fp:
        output_fp.write(attachment_fp.read())


email: Fwd Notas Magé 2.msg, email_path: pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails/Fwd Notas Magé 2.msg
10
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/12 - Ótica do Cizinho.pdf
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/38 EDSON JOSE OLIVEIRA DOS SANTOS.pdf
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/FLEXPRIN.pdf
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/NF 688 - ConsoR Zadar- Engetecnica (ENZA).pdf
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/1127 fogo em 20-07-23.pdf
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/Nota Fiscal Eletrônica 1717 - BALL - BVI.PDF
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/NF 689- PMMacae- 8ª Med CO-22-22 TB Contrato 045-22.pdf
pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/Nota Fiscal_229_Agosto-S

<h3 Salvando os attachments do e-mail </h3>

<mark> <b>2.</b> Extraindo compressed files </mark>

In [14]:
# Diretório onde você quer salvar os arquivos extraídos
output_dir = os.path.join(documentos_extracao_path, batch_name)

folder_file_dict = {}

for root, dirs, files in os.walk(msg_attachment_zip):
    #print(f'{root}  | {dirs} | {document} | {files}\n')
    
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.zip'):
            zip_file = file
            zip_file_path = os.path.join(root, file)
            # Obtém o nome base do arquivo ZIP para usar como subdiretório
            zip_base_name = os.path.splitext(os.path.basename(zip_file_path))[0]
            zip_basename = conv_filename(zip_base_name)
            
            # Cria o subdiretório com base no nome do arquivo ZIP
            root_output_dir = os.path.join(output_dir, zip_basename)
            
            if not os.path.exists(root_output_dir):
                os.makedirs(root_output_dir) # estou criando o diretorio caso nao exista

            # Abre o arquivo ZIP
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                for member in zip_ref.namelist():
                    # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
                    parts = member.rsplit('\\', 1)
                    folder_name = parts[0] if len(parts) > 1 else ''
                    #folder_name = conv_filename(folder_temp)
                    filename = parts[-1]
                    if filename:  # ignora diretórios
                        # Adiciona ao dicionário
                        #filename = conv_filename(filename)
                        # Cria um subdiretório se ele não existir
                        sub_dir = os.path.join(root_output_dir, folder_name)
                        if not os.path.exists(sub_dir):
                            os.makedirs(sub_dir)

                        # Salva o arquivo no subdiretório especificado
                        source = zip_ref.open(member)
                        target_path = os.path.join(sub_dir, filename)
                        
                        with open(target_path, "wb") as target:
                            target.write(source.read())
                            dir_path = os.path.dirname(filename)
                            
        elif file.lower().endswith('.rar'):
            rar_file = file
            rar_file = conv_filename_no_ext(rar_file)
            rar_file_path = os.path.join(root, file) 
            root_output_dir = os.path.join(output_dir, rar_file)
            if not os.path.exists(root_output_dir):
                os.makedirs(root_output_dir)
            Archive(rar_file_path).extractall(root_output_dir)  
            
        elif file.lower().endswith('.7z'):
            sevenz_file = file
            sevenz_file = conv_filename_no_ext(sevenz_file)
            sevenz_file_path = os.path.join(root, file)
            root_output_dir = os.path.join(output_dir, sevenz_file)                      
                            
            with py7zr.SevenZipFile(sevenz_file_path, mode='r') as z:
                z.extractall(root_output_dir)
                
        elif file.lower().endswith('.pdf'):
            file_path = os.path.join(root, file)
            new_path_name = os.path.join(output_dir, file)
            if not os.path.exists(output_dir):
                            os.makedirs(output_dir)
            shutil.move(file_path, new_path_name)
            
      

In [15]:
# Lista para armazenar as linhas
rows_list = []

# Função recursiva para adicionar linha
def add_row_recursively(rows_list, 
                        index, 
                        Batch, 
                        diretorio_ori, 
                        arquivo_origem, 
                        arquivo_destino, 
                        data_hora, 
                        tipo_doc_pdf,
                        trully_searcheable, 
                        qtd_paginas,
                        amostra_texto
                        ):
    if index == 0:
        return rows_list
    else:
        new_row = {
                    'index': index,
                    'Batch': Batch,
                    'diretorio_origem': diretorio_ori,
                    'nome_arquivo_origem': arquivo_origem,
                    'nome_arquivo_destino': arquivo_destino,
                    'data_processamento': data_hora,
                    'tipo_pdf': tipo_doc_pdf,
                    'trully_search': trully_searcheable,
                    'qut_paginas': qtd_paginas,
                    'amostra_text': amostra_texto
                    }
        rows_list.append(new_row)
        
        return add_row_recursively(rows_list, index-1, Batch, diretorio_ori, arquivo_origem, arquivo_destino, data_hora, tipo_doc_pdf, qtd_paginas)

<mark> <b>3.</b> Ajustando o filename e criando o dicionario </mark>

In [28]:

# Ajustando o filename e criando o dicionario
folder_file_dict = {}
rows_list = []
text_cabecalho = []
output_dir = os.path.join(documentos_extracao_path, batch_name)
i = 0
for root, dirs, files in os.walk("pipeline_extracao_documentos/2_documentos_para_extracao"):
    folder_name = os.path.basename(root)
    
    for file in files:
        file_path = os.path.join(root, file)
        #pesquisavel, metadados, paginas = is_pdf_searchable_analise(file_path)
        
        pdf_realmente_pesquisavel = confirma_pdf_pequisavel(file_path)

        new_name = conv_filename(file)
        new_path_name = os.path.join(root, new_name)
        #print(f'\nfile: {file} | new_name: {new_name} ')
        shutil.move(file_path, new_path_name)
        folder_file_dict.setdefault(folder_name, []).append(new_name)
        time_now = cron.timenow_pt_BR()
        new_row = {
                    'index': i,
                    'Batch': batch_name,
                    'diretorio_origem': folder_name,
                    'nome_arquivo_origem': file,
                    'nome_arquivo_destino': new_name,
                    'data_processamento': time_now,
                    'tipo_pdf': "pesquisavel",
                    'trully_search':pdf_realmente_pesquisavel,
                    'qut_paginas': paginas,
                    'amostra_text': "texto_cabe"
                    
                    }
        rows_list.append(new_row)
    i += 1

Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document


In [41]:
df_files = pd.DataFrame(rows_list)

In [42]:
df_files_v2 = pd.DataFrame(rows_list)

In [43]:
df_files_v2

,index,Batch,diretorio_origem,nome_arquivo_origem,nome_arquivo_destino,data_processamento,tipo_pdf,trully_search,qut_paginas
0,3,Batch_17,1002479,nota_fiscal_eletronica_190.pdf,nota_fiscal_eletronica_190.pdf,01/09/2023 18:44:46,True,True,1
1,3,Batch_17,1002479,nota_fiscal_eletronica_188.pdf,nota_fiscal_eletronica_188.pdf,01/09/2023 18:44:46,True,True,1
2,3,Batch_17,1002479,nota_fiscal_eletronica_189.pdf,nota_fiscal_eletronica_189.pdf,01/09/2023 18:44:46,True,True,1
3,4,Batch_17,1001924,listagem_de_nfs_e___sintetico_gf_pereira.pdf,listagem_de_nfs_e___sintetico_gf_pereira.pdf,01/09/2023 18:44:46,True,True,1
4,5,Batch_17,1004403,nf_11_igreja_evangelica_batista_eterna_vida.pdf,nf_11_igreja_evangelica_batista_eterna_vida.pdf,01/09/2023 18:44:46,True,True,1
5,6,Batch_17,1004463,nota_fiscal_coco_legal_julho_2023.pdf,nota_fiscal_coco_legal_julho_2023.pdf,01/09/2023 18:44:46,True,True,1
6,7,Batch_17,1004268,nota_fiscal_eletronica_julho.pdf,nota_fiscal_eletronica_julho.pdf,01/09/2023 18:44:46,True,True,1
7,7,Batch_17,1004268,relatorio_de_listagem.pdf,relatorio_de_listagem.pdf,01/09/2023 18:44:46,True,True,1
8,8,Batch_17,1005516,listagem_de_nfs_e__clinica_anna_stofoles.pdf,listagem_de_nfs_e__clinica_anna_stofoles.pdf,01/09/2023 18:44:46,True,True,1
9,9,Batch_17,11789914,nf_12_renilda.pdf,nf_12_renilda.pdf,01/09/2023 18:44:46,True,True,1


In [136]:
# Anexando o novo DataFrame ao original
df_extracao_files = df_extracao_files.append(df_files, ignore_index=True)

/tmp/ipykernel_9286/2817542823.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_extracao_files = df_extracao_files.append(df_files, ignore_index=True)


In [137]:
df_extracao_files

,index,Batch,diretorio_origem,nome_arquivo_origem,nome_arquivo_destino,data_processamento,tipo_pdf,qut_paginas
0,2,Batch_12,notas_mage_31082023,NF 689- PMMacae- 8ª Med CO-22-22 TB Contrato 0...,nf_689__pmmacae__8a_med_co_22_22_tb_contrato_0...,01/09/2023 14:22:44,False,1
1,2,Batch_12,notas_mage_31082023,nfse 4414.pdf,nfse_4414.pdf,01/09/2023 14:22:44,True,1
2,2,Batch_12,notas_mage_31082023,nfse 4409.pdf,nfse_4409.pdf,01/09/2023 14:22:44,True,1
3,2,Batch_12,notas_mage_31082023,NF 688 - ConsoR Zadar- Engetecnica (ENZA) (1).pdf,nf_688___consor_zadar__engetecnica_enza_1.pdf,01/09/2023 14:22:44,False,1
4,2,Batch_12,notas_mage_31082023,12225 COFIM -.pdf,12225_cofim__.pdf,01/09/2023 14:22:44,True,1
...,...,...,...,...,...,...,...,...
78,4,Batch_16,159871,2023 -8.pdf,2023__8.pdf,01/09/2023 15:28:23,True,1
79,5,Batch_16,160014,31-07.pdf,31_07.pdf,01/09/2023 15:28:23,True,1
80,5,Batch_16,160014,ACFrOgBLgYewSPQAweUd3QJkpDqN5Kp2dFIyNq7d6wJCRY...,acfrogblgyewspqaweud3qjkpdqn5kp2dfiynq7d6wjcry...,01/09/2023 15:28:23,True,1
81,6,Batch_16,126623,41C46D8F-73AB-4906-A4C6-C7DC92C05828.PDF,41c46d8f_73ab_4906_a4c6_c7dc92c05828.pdf,01/09/2023 15:28:23,True,1


In [138]:
folder_file_dict

{'115964': ['livro_de_registro_do_issqn.pdf'],
 '159871': ['2023__5.pdf',
  '2023__7.pdf',
  '2023__4.pdf',
  '2023__6.pdf',
  '2023__3.pdf',
  '2023__8.pdf'],
 '160014': ['31_07.pdf',
  'acfrogblgyewspqaweud3qjkpdqn5kp2dfiynq7d6wjcrymgxkby0xaq7m2xyrrh8asjkxsfk1z9f4bsqat1di5gppkc3ahrhnhavaawbjuamkpiluuxpydd2ovrxzk.pdf'],
 '126623': ['41c46d8f_73ab_4906_a4c6_c7dc92c05828.pdf'],
 '138565': ['b4066c58_f309_42e4_a992_55eb8961211e.pdf']}

### 2. Dicionarios e Dataframe

In [139]:
folder_names = []
file_counts = []
file_names = []
# Iterar sobre o dicionário para coletar informações
for folder, files in folder_file_dict.items():
    folder_names.append(folder)
    file_counts.append(len(files))
    file_names.append(files)
# folder_file_dict é algo como {'pasta1': 'arquivo1.pdf', 'pasta2': 'arquivo2.pdf'}
folder_names = list(folder_file_dict.keys())
file_names = list(folder_file_dict.values())    

Tamanho de folder_names: 5
Tamanho de file_counts: 5
Tamanho de file_names: 5


In [141]:
file_counts

[1, 6, 2, 1, 1]

In [142]:
dfs = {} 
# Criar o DataFrame do batch
df_batch = pd.DataFrame({
    "Dt_hora": [date_email],
    "Assunto": [msg_subject],
    "Arquivos_zip": [arquivos_zip],
    "Quantidade de Documentos": [file_counts],
    "De": [msg_sender],
    "batch": [batch_name],
    "email": [msg_email_address],

})


In [143]:
# Anexando o novo DataFrame ao original
df_processamento = df_processamento.append(df_batch, ignore_index=True)

/tmp/ipykernel_9286/1808731608.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_processamento = df_processamento.append(df_batch, ignore_index=True)


In [144]:
df_processamento

,Dt_hora,Assunto,Quantidade de Documentos,De,batch,email,Arquivos_zip
0,17/08/2023 16:11:50,Notas Magé,[115],Bruna Maciel Adame,Batch_1,bruna@modernizacaopublica.com.br,NaN
1,17/08/2023 15:49:21,Notas Sao Pedro da Aldeia,[28],Bruna Maciel Adame,Batch_2,bruna@modernizacaopublica.com.br,NaN
2,17/08/2023 11:56:51,Notas São Pedro da Aldeia,[10],Bruna Maciel Adame,Batch_3,bruna@modernizacaopublica.com.br,NaN
3,17/08/2023 12:01:16,Notas Mesquita,[2],Bruna Maciel Adame,na,bruna@modernizacaopublica.com.br,NaN
4,12/08/2023 11:25:42,Fwd: Notas Magé,[9],Verlânio Gallindo,Batch_5,verlanio@gmail.com,NaN
5,2023-10-08 23:20:56,Fwd: Notas Magé 2,[11],Verlânio Gallindo,Batch_6,verlanio@gmail.com,NaN
6,10/08/2023 23:20:56,Fwd: Notas Magé 2,[75],Verlânio Gallindo,Batch_6_Atualizado,verlanio@gmail.com,NaN
7,10/08/2023 17:54:14,Fwd: Notas Magé 1,"[21, 8, 18, 3]",Verlânio Gallindo,Batch_7,verlanio@gmail.com,"['07-2023 (1).7z', 'fwdrenotafiscal.zip', 'fwd..."
8,12/08/2023 11:25:42,Fwd: Notas Magé,[9],Verlânio Gallindo,Batch_8,verlanio@gmail.com,[]
9,31/08/2023 22:27:59,Notas Magé,"[2, 17]",Bruna Maciel Adame,Batch_12,bruna@modernizacaopublica.com.br,[]


### Salvar

In [7]:
# 1. Tratando nome de carga do df_processamento
dataset_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"

df_processamento_file = "df_processamento_"

df_processamento_file_write = df_processamento_file + str(i_test) + ".xlsx"

# 2. Tratando nome de carga do df_extracao_files_Batch
df_extracao_files_Batch_file = "df_extracao_files_Batch_"

df_extracao_files_Batch_file_write = df_extracao_files_Batch_file + str(i_test) + ".xlsx"


df_processamento_file_write_path = os.path.join(dataset_path, df_processamento_file_write)

df_processamento_file_write_path

df_extracao_files_Batch_file_write_path = os.path.join(dataset_path, df_extracao_files_Batch_file_write)

df_extracao_files_Batch_file_write_path

# Salvando o DF para excel
df_extracao_files.to_excel(df_extracao_files_Batch_file_write_path, index=False)

# Salvando o DF para excel
df_processamento.to_excel(df_processamento_file_write_path, index=False)

## funcoes modules

In [29]:
# 1. Funcao de conversao e resize do documento
def convertResize(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name


def convertResizeAnalise_1page(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    resized_pages = []
    for page in pages:
        resized_page = page.resize((2067, 2923))
        resized_pages.append(resized_page)
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name


# Funcao de conversao e resize do documento
def convertResize_analise(nome_documento, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(nome_documento)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name



# Trata Ocr
def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number2(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores


# 3. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 



# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# 1. Interacao para pesquisar prefeitura
def pesquisa_texto(texto):
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', texto)
    if nome_prefeitura_match:
        is_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        
        return  is_prefeitura
    else:
        raise ValueError("Nao consegui pesquisar")

    
    
# 2. Extracao OCR
def extract_text_from_coordinates(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

# 1. funçao basica de modelo 
def executa_model_frame(model, secao, father_name):

    data_dados_frame = {}
    
    tipo = "frame"
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['label'] == f_frame_name) & (frames_nf_v4_df['type'] == tipo)]

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_frame = extract_text_from_coordinates(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        frame_seq = row_frame['seq']
        frame_model = row_frame['model']
        frame_label = row_frame['label']
        frame_type = row_frame['type']
        frame_section = row_frame['section_json']
        frame_reference = row_frame['reference']
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        #print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
        
    return extracted_text_frame



# Sao iguais 
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text 



# Trata texto extraido
# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None
    
    
# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)



def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited




#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None
    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        
#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  


    
def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):

    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz           
    #print(resultado_extraido)
    return resultado_extraido






def cabecalho_prefeitura():
    valor_dict = {}
    dados_prefeitura = {}
    f_frame_name = "1_frame_prefeitura_nf"
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)
    text_splited = texto.split('\n')
    
    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
    if valor_dict:
        dados_prefeitura.update(valor_dict)
        
        
    return dados_prefeitura 
                
def cabecalho_dados():

    valor = {}   
    f_frame_name = "1_frame_dados_nf"
    
    dadinho_dados_nf = {}
    
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)    
    text_splited = texto_extraido(texto)
    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

    string_pesquisa = "Número da Nota:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
    dadinho_dados_nf['numero_nota_fiscal'] = texto


    string_pesquisa = "Competência:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['competencia'] = texto
    
    
    string_pesquisa = "ata e Hora da Emissão:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['dt_hr_emissao'] = texto
    
    
    string_pesquisa = "Código Verificação:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['codigo_verificacao'] = texto
    
    return dadinho_dados_nf   


def extract_fields_prestador_cnpj(text): # Função para extrair campos e valores dentro de um retângulo
    
    
    nf_data_prestador_cnpj = {}
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))


    # Extrair Telefone
    telefone_str = None
    
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador_cnpj['telefone'] = telefone_str
    else:
        nf_data_prestador_cnpj['telefone'] = None   
    
    
    return nf_data_prestador_cnpj 


def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj 

def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


def extract_dados_comple_obs(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            #print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            
            if frame_father == "5_frame_dados_complementares":
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['section'] = section
                
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
                if text == '':
                    text = None
                    nf_data_dados_complementares['dados_complementares'] = text
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                    
                return nf_data_dados_complementares                
                
            elif frame_father == "5_frame_observacao":
                nf_data_observacao = {}
                nf_data_observacao['section'] = section 
                                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', extracted_text_box, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                return nf_data_observacao 
            
            
secao = "1 - CABECALHO"
f_frame_name = "1_frame_prefeitura_nf"


#4. Extrai prefeitura
def extract_prefeitura(model, father, values):
    
    tipo = "sframe_field"
    data_extrated_prefeitura = {}
    #print(tipo)

    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo)]

    for index_sframe, row_sframe in filtered_frames_nf_v4_df.iterrows():
        
        label_value = row_sframe['label']
        
        #print("label_value", label_value)
        
        if label_value == "nome_prefeitura":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
        elif label_value == "secretaria":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result) 
        elif label_value == "tipo_nota_fiscal":
            reference_value = row_sframe['reference']  
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
                    
    return data_extrated_prefeitura




def processa_cnae_outros(text):
    nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
    if nf_data_CNAE_match:
        try:
            # Remove a primeira ocorrência de "CNAE:"
            nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
            # Remover quebras de linha
            nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
            return nf_data_CNAE_str 
        except Exception as e:
            print(f"Erro busca cnae: {e}") 
        
    return None   



def extract_fb_outras_inf(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == model)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        
        string_pesquisa = row_field['reference']
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        label = row_field['label']
        #print(f'extracted_text_box {extracted_text_box}, label {label}')
        text = extracted_text_box.replace('\n', '')
        if text.startswith(string_pesquisa):
            #print("aqui:", text)
            text = text[len(label):].strip()
            data_box_valores[label] = text
    
    return   data_box_valores      
                 
    

## | REALMENTE O PIPELINE

In [30]:
def processa_raster_pdf(image_2work):
    
    secao = "1 - CABECALHO"
    try:
        nro_nota = 0
        nd_data_cabecalho = {}
        nd_data_cabecalho['secao'] = secao
        valor_dict = {}
        dados_prefeitura = {}
        f_frame_name = "1_frame_prefeitura_nf"
        # 1. funçao basica de modelo 
        texto = executa_model_frame(model, secao, f_frame_name)
        text_splited = texto.split('\n')
        
        valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
        if valor_dict:
            dados_prefeitura.update(valor_dict)
        valor = {}   
        f_frame_name = "1_frame_dados_nf"
        dadinho_dados_nf = {}
        # 1. funçao basica de modelo 
        texto = executa_model_frame(model, secao, f_frame_name)    
        text_splited = texto_extraido(texto)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        dadinho_dados_nf['numero_nota_fiscal'] = texto
        nro_nota = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dadinho_dados_nf['competencia'] = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dadinho_dados_nf['dt_hr_emissao'] = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dadinho_dados_nf['codigo_verificacao'] = texto
        
        nd_data_cabecalho.update(dados_prefeitura)
        nd_data_cabecalho.update(dadinho_dados_nf)
    except Exception as e:
        # erros_cabecalho = {}
        err_msg = f"Erro de processo cabecalho: {e}"
        print(err_msg)
        # erros['documento'] = file
        # erros_cabecalho['secao'] = secao
        # erros_cabecalho['erro'] = err_msg
        # erros.update(erros_cabecalho)                
    
    secao = "2. PRESTADOR DE SERVIÇO"
    try:
        f_frame_name = "2_frame_cnpj_prestador"
        nd_data_prestador = {}
        prestador_inscricao = {}
        nd_data_prestador['secao'] = secao
        prestador_cnpj_value = {}
        texto = executa_model_frame(model, secao, f_frame_name)    
        #text_splited = texto_extraido(texto)
        prestador_cnpj_value = extract_fields_prestador_cnpj(texto)
        if prestador_cnpj_value:
            nd_data_prestador.update(prestador_cnpj_value)
    except Exception as e:
        # erros_cnpj_prestador = {}
        err_msg = (f"Erro prestador cnpj: {e}")
        print(err_msg)
        
        # erros_cnpj_prestador['secao'] = secao
        # erros_cnpj_prestador['erro'] = err_msg
        # erros.update(erros_cnpj_prestador)       
        
    try:
        f_frame_name = "2_frame_inscricao_prestador" 
        texto = executa_model_frame(model, secao, f_frame_name)    
        text_splited = texto_extraido(texto)
        keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
        string_pesquisa = "Inscrição Municipal:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        prestador_inscricao['prestador_inscricao'] = texto
        
        string_pesquisa = "Inscrição Estadual:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        prestador_inscricao['inscricao_estadual'] = texto
        nd_data_prestador.update(prestador_inscricao)
    except Exception as e:
        # erros_inscricao_prestador = {}
        err_msg = (f"Erro de processo inscricao prestador: {e}")
        print(err_msg)
        # erros_inscricao_prestador['secao'] = secao
        # erros_inscricao_prestador['erro'] = err_msg
        # erros.update(erros_inscricao_prestador)

    try:
        f_frame_name = "2_frame_dados_prestador"
        prestador_dados_value = {}
        
        keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
        string_pesquisa = "Nome/Razão Social:"
        texto = executa_model_frame(model, secao, f_frame_name)
        text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        prestador_dados_value['razao_social'] = texto

        string_pesquisa = "Nome de Fantasia:"
        #texto = executa_model_frame(model, secao, f_frame_name)
        #text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        prestador_dados_value['nome_fantasia'] = texto
        
        string_pesquisa = "Endereço:"
        #texto = executa_model_frame(model, secao, f_frame_name)
        #text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        prestador_dados_value['endereco'] = texto
        
        string_pesquisa = "E-mail:"
        #texto = executa_model_frame(model, secao, f_frame_name)
        #text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        prestador_dados_value['email'] = texto
        nd_data_prestador.update(prestador_dados_value)
        
    except Exception as e:
        print(f"Erro prestador dados: {e}") 
        
    
    secao = "3. TOMADOR DE SERVIÇO"
    try:
        nd_data_tomador = {}
        tomador_cnpj_value = {}
        nd_data_tomador['secao'] = secao
        f_frame_name = "3_frame_cnpj_tomador"
        texto = executa_model_frame(model, secao, f_frame_name)    
        text_splited = texto_extraido(texto) 
        tomador_cnpj_value = extract_fields_tomador_cnpj(texto)
        if tomador_cnpj_value:
            nd_data_tomador.update(tomador_cnpj_value)                  
    except Exception as e:
        print(f"Erro tomador cnpj: {e}")
        
    f_frame_name = "3_frame_inscricao_tomador"    
    try:
        data_tomador_inscricao = {}
        keyword_list = ['RG:', 'Inscrição Estadual:']
        string_pesquisa = "RG:"
        texto = executa_model_frame(model, secao, f_frame_name)    
        text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_tomador_inscricao['rg'] = texto
        
        string_pesquisa = "Inscrição Estadual:"
        #texto = executa_model_frame(model, secao, f_frame_name)    
        #text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_tomador_inscricao['inscricao_estadual'] = texto
        nd_data_tomador.update(data_tomador_inscricao)
    except Exception as e:
        print(f"Erro tomador inscricao: {e}")
        
    f_frame_name = "3_frame_dados_tomador"
    try: 
        data_tomador_dados = {}   
        keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']
        string_pesquisa = "Nome/Razão Social:"
        texto = executa_model_frame(model, secao, f_frame_name)    
        text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_tomador_dados['razao_social'] = texto
        
        string_pesquisa = "Endereço:"
        #texto = executa_model_frame(model, secao, f_frame_name)    
        #text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_tomador_dados['endereco'] = texto
        
        string_pesquisa = "E-mail"
        #texto = executa_model_frame(model, secao, f_frame_name)    
        #text_splited = texto_extraido(texto)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_tomador_dados['email'] = texto
        
        nd_data_tomador.update(data_tomador_dados)
    except Exception as e:
        print(f"Erro tomador dados: {e}") 
        
    secao = "4. DESCRIMINACAO DOS SERVIÇOS"
    try:
        nd_data_servico = {}
        nd_data_servico['secao'] = secao
        f_frame_name = "4_frame_descricao_totais"
        texto = executa_model_frame(model, secao, f_frame_name)
        text = texto.replace('\n', ' ')
        label = "DISCRIMINAÇÃO DOS SERVIÇOS"
        if text.startswith(label):
            text = text[len(label):].strip()
        nd_data_servico['discriminacao_servicos'] = text 

    except Exception as e:
        print(f"Erro descricao servico: {e}")
            
    secao = "5. VALOR TOTAL"
    try:
        nd_data_valor_total = {}
        data_valor_total['secao'] = secao
        f_frame_name = "4_frame_valor_total"   
        text = executa_model_frame(model, secao, f_frame_name)  
        valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
        if valor_total_match:
            valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
            nd_data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
    except Exception as e:
        print(f"Erro valor total: {e}")

    secao = "6. CNAE e Item da Lista de Serviços"
    try:
        nd_data_CNAE = {}
        nd_data_CNAE['secao'] = secao
        f_frame_name = "4_frame_cnae_itens_servico"   
        Texto_extraido = executa_model_frame(model, secao, f_frame_name)
        text_splited = Texto_extraido.split('\n')
        # Processando CNAE
        cnae_line = [line for line in text_splited if 'CNAE' in line][0]
        cnae_number = int(extract_number(cnae_line))
        cnae_value = cnae_dict.get(cnae_number, "Valor não encontrado")
        if cnae_value == 'Valor não encontrado':
            cnae_value = processa_cnae_outros(cnae_line)
            cnae_value = cnae_value.upper()
            nd_data_CNAE['cnae'] = cnae_value
        else:
            cnae_value = cnae_value.upper()
            cnae_value = str(cnae_number) + " - " + cnae_value
            nd_data_CNAE['cnae'] = cnae_value
            nd_data_CNAE['item_lista_servicos'] = item_servico_value
    except Exception as e:
        print(f"Erro busca cnae: {e}")    

    try:
        item_servico_line = [line for line in text_splited if 'Item da Lista de Serviços' in line][0]
        item_servico_number = float(extract_number(item_servico_line))
        item_servico_value = item_servico_dict.get(item_servico_number, "Valor não encontrado")
        item_servico_value = item_servico_value.upper()
        item_servico_value = str(item_servico_number) + " - " + item_servico_value
        nd_data_CNAE['item_lista_servicos'] = item_servico_value
    except Exception as e:
        print(f"Erro busca Itens de servico: {e}")  

    secao = "8. DADOS COMPLEMENTARES"
    try:
        nd_data_valores = {}
        nd_data_valores['secao'] = secao
        f_frame_name = "5_frame_valores_impostos"   
        
        result = extract_fields_box(model, f_frame_name, secao)
        if result:
            nd_data_valores.update(result)

        # secao: 8 - DADOS COMPLEMENTARES"
        nd_data_dados_complementares = {}
        f_frame_name  = "5_frame_dados_complementares"
        section = "8. DADOS COMPLEMENTARES"

        nd_data_dados_complementares = extract_dados_comple_obs(model, f_frame_name, section)                                           
                                
                                
        # secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
        nd_data_outras_informacoes = {}
        father_value = "5_frame_inf_criticas"
        section = "9. OUTRAS INFORMAÇOES / CRITICAS"
        
        result = extract_fb_outras_inf(model, father_value, section)
        if result:
            nd_data_outras_informacoes.update(result)                        
                            
        # secao: 10. OBSERVACOES
        nd_data_observacao = {}
        f_father = "5_frame_observacao"
        section = "10. OBSERVACOES"

        nd_data_observacao = extract_dados_comple_obs(model, f_father, section)
    except Exception as e:
        print(f"Erro valores complementares:")  
        
    
    return nro_nota, nd_data_cabecalho, nd_data_prestador, nd_data_tomador, nd_data_servico, nd_data_valor_total, nd_data_CNAE, nd_data_valores, nd_data_dados_complementares, nd_data_outras_informacoes, nd_data_observacao      

In [31]:
def processa_pdf_pesquisavel(file_path):
    
   
    status = "O PDF é pesquisável"
    # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    print(text)
    
    if text:
       page_number = 0
       print(page_number)
    else:
       page_number = 1
       print(page_number)
    
    nf_data_cabecalho = {}
    
    page = pdf_document[page_number]
    x0 = 0
    y0 = 0
    x1 = 600
    y1 = 110

    text = page.get_text("text", clip=(x0, y0, x1, y1))
    nf_data_cabecalho = Extc.extract_fields_cabecalho(text)
    #nf_data_cabecalho = extract_fields_cabecalho(text)
    
    nro_nota = nf_data_cabecalho['numero_nota_fiscal']

        
    
    
    
    # 2. PRESTADOR DE SERVIÇO
    # Definir retângulo de interesse
    nf_data_prestador = {}
    x0 = 0
    y0 = 100
    x1 = 600
    y1 = 236  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    nf_data_prestador = Extc.extract_fields_prestador(text)
    #nf_data_prestador = extract_fields_prestador(text)
    
    # 3. TOMADOR DE SERVIÇO
    # Definir retângulo de interesse
    nf_data_tomador = {}
    x0 = 0
    y0 = 210
    x1 = 600
    y1 = 340  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    
    nf_data_tomador = Extc.extract_fields_tomador(text)
    #nf_data_tomador = extract_fields_tomador(text)
    
    
    # 4. DESCRIMINACAO DOS SERVIÇOS
    nf_data_servico = {}
    nf_data_servico['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"
    # Definir retângulo de interesse
    x0 = 0
    y0 = 330
    x1 = 600
    y1 = 500  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))

    # Remover quebras de linha e rótulo
    text = text.replace('\n', ' ')
    label = "DISCRIMINAÇÃO DOS SERVIÇOS"
    if text.startswith(label):
        text = text[len(label):].strip()

    # Atribuir texto ao dicionário
    nf_data_servico['discriminacao_servicos'] = text
    
    
    # 5. VALOR TOTAL
    nf_data_valor_total = {}
    nf_data_valor_total['secao'] = "5. VALOR TOTAL"

    # Definir retângulo de interesse
    x0 = 0
    y0 = 500
    x1 = 600
    y1 = 550  # Ajuste este valor para delimitar a região vertical


    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))

    # Usar expressão regular para extrair apenas os caracteres numéricos e pontos decimais
    valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
    if valor_total_match:
        valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
        nf_data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
        
        
    
    
    # 6. CNAE e Item da Lista de Serviços
    nf_data_CNAE = {}
    nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"

    # Definir retângulo de interesse CNAE
    x0 = 0
    y0 = 530
    x1 = 600
    y1 = 540  # Ajuste este valor para delimitar a região vertical


    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))


    # Extrair CNAE
    nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
    if nf_data_CNAE_match:
        # Remove a primeira ocorrência de "CNAE:"
        nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
        # Remover quebras de linha
        nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
        nf_data_CNAE['cnae'] = nf_data_CNAE_str
        
    
    
    # Item da Lista de Serviços    
    # Definir retângulo de interesse - Item da Lista de Serviços
    x0 = 0
    y0 = 545
    x1 = 600
    y1 = 560  # Ajuste este valor para delimitar a região vertical    
    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))     
        
    # Extrair Item da Lista de Serviços
    nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
    if nf_item_lista_servicos_match:
        nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1) 
        # Remover quebras de linha
        #nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n \n', '')
        nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n', ' ')
        nf_data_CNAE['item_lista_servicos'] = nf_item_lista_servicos_str
        
    
    # 7. VALORES E IMPOSTOS
    # Definir retângulo de interesse
    nf_data_valores = {}
    
    x0 = 0
    y0 = 550
    x1 = 600
    y1 = 650  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))

    # Extrair campos e valores
    nf_data_valores = Extc.extract_fields_impostos(text)
    #nf_data_valores = extract_fields_impostos(text)
    
    # 8. DADOS COMPLEMENTARES
    nf_data_dados_complementares = {}
    nf_data_dados_complementares['secao'] = "8. DADOS COMPLEMENTARES"

    # Definir retângulo de interesse
    x0 = 0
    y0 = 650
    x1 = 600
    y1 = 680  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    # Remove a primeira ocorrência de "Observação:"
    text = re.sub(r'^DADOS COMPLEMENTARES', '', text, count=1)
    if text == " ":
        text = "NONE"
        nf_data_dados_complementares['dados_complementares'] = text.strip()
    else:    
        # Extrair texto dentro do retângulo
        nf_data_dados_complementares['dados_complementares'] = text.strip()
    
    
    
    # 9. OUTRAS INFORMAÇOES / CRITICAS  
    # Definir retângulo de interesse
    nf_data_outras_informacoes = {}
    x0 = 0
    y0 = 680
    x1 = 600
    y1 = 725  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))

    # Extrair campos e valores
    nf_data_outras_informacoes = Extc.extract_fields_outras_info(text)
    #nf_data_outras_informacoes = extract_fields_outras_info(text)
    
    
    # 10. OBSERVACOES
    nf_data_observacao = {}
    nf_data_observacao['secao'] = "10. OBSERVACOES"
    # Definir retângulo de interesse
    x0 = 0
    y0 = 725
    x1 = 600
    y1 = 760  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))

    # Remove a primeira ocorrência de "Observação:"
    text = re.sub(r'^Observação:', '', text, count=1)

    # Remover quebras de linha
    text = text.replace('\n', ' ')

    # Extrair texto dentro do retângulo
    nf_data_observacao['observacao'] = text.strip()
    
    # try:
    #     nr_nro_nf = nro_nota
    # except Exception as e:
    #     print(f"Nao encontrado nro da NF: {e}")       
    

    return nro_nota, nf_data_cabecalho, nf_data_prestador, nf_data_tomador, nf_data_servico, nf_data_valor_total, nf_data_CNAE, nf_data_valores, nf_data_dados_complementares, nf_data_outras_informacoes, nf_data_observacao       

### 1. Processo RASTER PDF

In [34]:
erros = {}

# 1. Leitura recursiva de diretorios e arquivos a partir de root
pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

nf_data_servico = {}#VERIFICAR
analise_doc_nf = {} #VERIFICAR
file_data = [] #VERIFICAR

list_document_pages = []
#nro_nota = 0
# TEMP
# Nome do arquivo json
nome_formado_json = batch_name +".json"
#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)
#root_doc_analise = os.path.join(documentos_extracao_path, batch_name)
#print(root_doc_analise)
i = 1
for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    #print(dir_name)
    for file in files:
        
        if file.lower().endswith('.pdf'):
            doc2convert = file
            document_path_1 = os.path.join(root, file)
            pdf_document = fitz.open(document_path_1)
            #page_number = 0  # Defina o número da página que deseja analisar
            #page = pdf_document[page_number]
            
            documento_pdf = True
            pesquisavel, metadados, paginas = is_pdf_searchable_analise(document_path_1)
            
            
            pdf_realmente_pequisavel = confirma_pdf_pequisavel(document_path_1)
            
            if not pdf_realmente_pequisavel:
                print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                processo = "PDF_Raster"
                image_2work, name_image_2work = convertResizeAnalise_1page(file, document_path_1, image_resized_path)    
                
                
                secao = "1 - CABECALHO"
                try:
                    nro_nota = 0
                    data_cabecalho = {}
                    data_cabecalho['secao'] = secao
                    valor_dict = {}
                    dados_prefeitura = {}
                    f_frame_name = "1_frame_prefeitura_nf"
                    # 1. funçao basica de modelo 
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text_splited = texto.split('\n')
                    
                    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
                    if valor_dict:
                        dados_prefeitura.update(valor_dict)
                    valor = {}   
                    f_frame_name = "1_frame_dados_nf"
                    dadinho_dados_nf = {}
                    # 1. funçao basica de modelo 
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

                    string_pesquisa = "Número da Nota:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
                    dadinho_dados_nf['numero_nota_fiscal'] = texto
                    nro_nota = texto
                    
                    string_pesquisa = "Competência:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['competencia'] = texto
                    
                    string_pesquisa = "ata e Hora da Emissão:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['dt_hr_emissao'] = texto
                    
                    string_pesquisa = "Código Verificação:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['codigo_verificacao'] = texto
                    
                    data_cabecalho.update(dados_prefeitura)
                    data_cabecalho.update(dadinho_dados_nf)
                except Exception as e:
                    erros_cabecalho = {}
                    err_msg = f"Erro de processo cabecalho: {e}"
                    erros['documento'] = file
                    erros_cabecalho['secao'] = secao
                    erros_cabecalho['erro'] = err_msg
                    erros.update(erros_cabecalho)                
               
                
                
                
                secao = "2. PRESTADOR DE SERVIÇO"
                try:
                    f_frame_name = "2_frame_cnpj_prestador"
                    data_prestador = {}
                    prestador_inscricao = {}
                    data_prestador['secao'] = secao
                    prestador_cnpj_value = {}
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    prestador_cnpj_value = extract_fields_prestador_cnpj(texto)
                    if prestador_cnpj_value:
                        data_prestador.update(prestador_cnpj_value)
                except Exception as e:
                    erros_cnpj_prestador = {}
                    err_msg = (f"Erro prestador cnpj: {e}")
                    erros_cnpj_prestador['secao'] = secao
                    erros_cnpj_prestador['erro'] = err_msg
                    erros.update(erros_cnpj_prestador)       
                    
                try:
                    f_frame_name = "2_frame_inscricao_prestador" 
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                    string_pesquisa = "Inscrição Municipal:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_inscricao['prestador_inscricao'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_inscricao['inscricao_estadual'] = texto
                    data_prestador.update(prestador_inscricao)
                except Exception as e:
                    erros_inscricao_prestador = {}
                    err_msg = (f"Erro de processo inscricao prestador: {e}")
                    erros_inscricao_prestador['secao'] = secao
                    erros_inscricao_prestador['erro'] = err_msg
                    erros.update(erros_inscricao_prestador)

                try:
                    f_frame_name = "2_frame_dados_prestador"
                    prestador_dados_value = {}
                    
                    keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['razao_social'] = texto

                    string_pesquisa = "Nome de Fantasia:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['nome_fantasia'] = texto
                    
                    string_pesquisa = "Endereço:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['endereco'] = texto
                    
                    string_pesquisa = "E-mail:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['email'] = texto
                    data_prestador.update(prestador_dados_value)
                    
                except Exception as e:
                    print(f"Erro prestador dados: {e}") 
                    
                
                secao = "3. TOMADOR DE SERVIÇO"
                try:
                    data_tomador = {}
                    tomador_cnpj_value = {}
                    data_tomador['secao'] = secao
                    f_frame_name = "3_frame_cnpj_tomador"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto) 
                    tomador_cnpj_value = extract_fields_tomador_cnpj(texto)
                    if tomador_cnpj_value:
                        data_tomador.update(tomador_cnpj_value)                  
                except Exception as e:
                    print(f"Erro tomador cnpj: {e}")
                    
                f_frame_name = "3_frame_inscricao_tomador"    
                try:
                    data_tomador_inscricao = {}
                    keyword_list = ['RG:', 'Inscrição Estadual:']
                    string_pesquisa = "RG:"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_inscricao['rg'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_inscricao['inscricao_estadual'] = texto
                    data_tomador.update(data_tomador_inscricao)
                except Exception as e:
                    print(f"Erro tomador inscricao: {e}")
                    
                f_frame_name = "3_frame_dados_tomador"
                try: 
                    data_tomador_dados = {}   
                    keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['razao_social'] = texto
                    
                    string_pesquisa = "Endereço:"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['endereco'] = texto
                    
                    string_pesquisa = "E-mail"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['email'] = texto
                    
                    data_tomador.update(data_tomador_dados)
                except Exception as e:
                    print(f"Erro tomador dados: {e}") 
                    
                secao = "4. DESCRIMINACAO DOS SERVIÇOS"
                try:
                    data_servico = {}
                    data_servico['secao'] = secao
                    f_frame_name = "4_frame_descricao_totais"
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text = texto.replace('\n', ' ')
                    label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                    if text.startswith(label):
                        text = text[len(label):].strip()
                    data_servico['discriminacao_servicos'] = text 

                except Exception as e:
                    print(f"Erro descricao servico: {e}")
                     
                secao = "5. VALOR TOTAL"
                try:
                    data_valor_total = {}
                    data_valor_total['secao'] = secao
                    f_frame_name = "4_frame_valor_total"   
                    text = executa_model_frame(model, secao, f_frame_name)  
                    valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
                    if valor_total_match:
                        valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                        data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                except Exception as e:
                    print(f"Erro valor total: {e}")

                secao = "6. CNAE e Item da Lista de Serviços"
                try:
                    data_CNAE = {}
                    data_CNAE['secao'] = secao
                    f_frame_name = "4_frame_cnae_itens_servico"   
                    Texto_extraido = executa_model_frame(model, secao, f_frame_name)
                    text_splited = Texto_extraido.split('\n')
                    # Processando CNAE
                    cnae_line = [line for line in text_splited if 'CNAE' in line][0]
                    cnae_number = int(extract_number(cnae_line))
                    cnae_value = cnae_dict.get(cnae_number, "Valor não encontrado")
                    if cnae_value == 'Valor não encontrado':
                        cnae_value = processa_cnae_outros(cnae_line)
                        cnae_value = cnae_value.upper()
                        data_CNAE['cnae'] = cnae_value
                    else:
                        cnae_value = cnae_value.upper()
                        cnae_value = str(cnae_number) + " - " + cnae_value
                        data_CNAE['cnae'] = cnae_value
                        data_CNAE['item_lista_servicos'] = item_servico_value
                except Exception as e:
                    print(f"Erro busca cnae: {e}")    
          
                try:
                    item_servico_line = [line for line in text_splited if 'Item da Lista de Serviços' in line][0]
                    item_servico_number = float(extract_number(item_servico_line))
                    item_servico_value = item_servico_dict.get(item_servico_number, "Valor não encontrado")
                    item_servico_value = item_servico_value.upper()
                    item_servico_value = str(item_servico_number) + " - " + item_servico_value
                    data_CNAE['item_lista_servicos'] = item_servico_value
                except Exception as e:
                    print(f"Erro busca Itens de servico: {e}")  

                secao = "8. DADOS COMPLEMENTARES"
                try:
                    data_valores = {}
                    data_valores['secao'] = secao
                    f_frame_name = "5_frame_valores_impostos"   
                    
                    result = extract_fields_box(model, f_frame_name, secao)
                    if result:
                        data_valores.update(result)
            
                    # secao: 8 - DADOS COMPLEMENTARES"
                    data_dados_complementares = {}
                    f_frame_name  = "5_frame_dados_complementares"
                    section = "8. DADOS COMPLEMENTARES"

                    data_dados_complementares = extract_dados_comple_obs(model, f_frame_name, section)                                           
                                            
                                            
                    # secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
                    data_outras_informacoes = {}
                    father_value = "5_frame_inf_criticas"
                    section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                    
                    result = extract_fb_outras_inf(model, father_value, section)
                    if result:
                        data_outras_informacoes.update(result)                        
                                        
                    # secao: 10. OBSERVACOES
                    data_observacao = {}
                    f_father = "5_frame_observacao"
                    section = "10. OBSERVACOES"

                    data_observacao = extract_dados_comple_obs(model, f_father, section)
                except Exception as e:
                    print(f"Erro valores complementares: {e}")   
                
                nome_arquivo = file
                #pdf_info["diretorio"] = os.path.basename(root)
                try:
                    pdf_info[nro_nota] = {
                    "dados_NF_PDF": {
                        "data_cabecalho": data_cabecalho,
                        "data_prestador": data_prestador,
                        "data_tomador": data_tomador,
                        "data_servico": data_servico,
                        "data_valor_total": data_valor_total,
                        "data_CNAE": data_CNAE,
                        "data_valores": data_valores,
                        "data_dados_complementares": data_dados_complementares,
                        "data_outras_informacoes": data_outras_informacoes,
                        "data_observacao": data_observacao,
                    },
                    "diretorio": dir_name, #os.path.basename(root)
                    "nome_arquivo": nome_arquivo,
                    "Batch": batch_name,
                    "modelo": model,
                    "processo": processo,
                }
                except Exception as e:
                    print(f"Erro ao gerar o json: {e}")
                pdf_document.close()
                
                #print(pdf_info)
                #if paginas == 1:
                if paginas > 1000:
                    if i == 1000: #Define quantidade de tratamento de documentos raster PDF
                        break
            i +=1 
processo = "Raster_PDF"
                
# Nome do arquivo json
nome_formado_json = batch_name + "_" + processo + ".json"

json_file_path = os.path.join(json_path, nome_formado_json )


# Salvando as informações em um arquivo JSON (novo formato nome arquivo V2)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")

Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
Erro ao abrir pagina do PDF: page not in document
As informações foram salvas em pipeline_extracao_documentos/5_documentos_processados/jsons/Batch_17_Raster_PDF.json


### 2. Processo PDF Pesquisavel

In [23]:
root_doc_analise = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

batch_name = "Batch_17"

In [36]:
erros = {}
# 1. Leitura recursiva de diretorios e arquivos a partir de root
pdf_info = {}  # Dicionário para armazenar informações sobre PDFs


data_cabecalho_final = {}
data_prestador_final = {}
data_tomador_final = {}
data_servico_final = {}
data_valor_total_final = {}
data_CNAE_final = {}
data_valores_final = {}
data_dados_complementares_final = {}
data_outras_informacoes_final = {}
data_observacao_final = {}

result_1 = {}
result_2 = {}
result_3 = {}
result_4 = {}
result_5 = {}
result_6 = {}
result_7 = {}
result_8 = {}
result_9 = {}
result_10 = {}

i = 1
for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    #print(dir_name)
    for file in files:
        
        if file.lower().endswith('.pdf'):
            doc2convert = file
            document_path_1 = os.path.join(root, file)
            pdf_document = fitz.open(document_path_1)
            #page_number = 0  # Defina o número da página que deseja analisar
            #page = pdf_document[page_number]

            
            documento_pdf = True
            pesquisavel, metadados, paginas = is_pdf_searchable_analise(document_path_1)
            print(f'\ndiretorio: {dir_name} nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
            
            pdf_realmente_pequisavel = confirma_pdf_pequisavel(document_path_1)
            
            print("pdf_realmente_pequisavel: ", pdf_realmente_pequisavel)
            
            if not pdf_realmente_pequisavel:
                print(f'\nRaster PDF diretorio: {dir_name} nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
               
 
            else:
                print(f'\nPDF Pesquisavel diretorio: {dir_name} nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                try:
                    processo = "PDF_Pesquisavel"


                    nro_nota, result_1, result_2, result_3, result_4, result_5, result_6, result_7, result_8, result_9, result_10 = processa_pdf_pesquisavel(document_path_1)
                    
                    if result_1:
                        data_cabecalho_final.update(result_1)
                    if result_2:
                        data_prestador_final.update(result_2)   
                    if result_3:
                        data_tomador_final.update(result_3)
                    if result_4:
                        data_servico_final.update(result_4)
                    if result_5:
                        data_valor_total_final.update(result_5)
                    if result_6:
                        data_CNAE_final.update(result_6)
                    if result_7:
                        data_valores_final.update(result_7)
                    if result_8:
                        data_dados_complementares_final.update(result_8)                                                                   
                    if result_9:
                        data_outras_informacoes_final.update(result_9)   
                    if result_10:
                        data_observacao_final.update(result_10)
                        
                except Exception as e:
                    print(f"Erro ao gerar dict nf: {e}")         
                
            nome_arquivo = file
            
            
            try:
                pdf_info[nro_nota] = {
                        "dados_NF_PDF": {
                        "data_cabecalho": data_cabecalho_final,
                        "data_prestador": data_prestador_final,
                        "data_tomador": data_tomador_final,
                        "data_servico": data_servico_final,
                        "data_valor_total": data_valor_total_final,
                        "data_CNAE": data_CNAE_final,
                        "data_valores": data_valores_final,
                        "data_dados_complementares": data_dados_complementares_final,
                        "data_outras_informacoes": data_outras_informacoes_final,
                        "data_observacao": data_observacao_final,
                    },
                    "diretorio": dir_name, #os.path.basename(root)
                    "nome_arquivo": nome_arquivo,
                    "Batch": batch_name,
                    "modelo": model,
                    "pdf_realmente_pequisavel": pdf_realmente_pequisavel,
                    "processo": processo,
                }
            except Exception as e:
                print(f"Erro ao gerar o json: {e}")
                        #print(pdf_info) 
                    
                                                 
            pdf_document.close()    
                    
            i += 1
            
            print("i: ", i) 
     
     
     
     
                

# Nome do arquivo json
nome_formado_json = batch_name + "_" + processo + ".json"

json_file_path = os.path.join(json_path, nome_formado_json )


# Salvando as informações em um arquivo JSON (novo formato nome arquivo V2)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")


diretorio: Batch_17 nro: 1 | doc: nf_689__pmmacae__8a_med_co_22_22_tb_contrato_045_22.pdf | pdf?: True | pesquisavel?: False | paginas: 1

Erro ao abrir pagina do PDF: page not in document
pdf_realmente_pequisavel:  (False, 1)

PDF Pesquisavel diretorio: Batch_17 nro: 1 | doc: nf_689__pmmacae__8a_med_co_22_22_tb_contrato_045_22.pdf | pdf?: True | pesquisavel?: False | paginas: 1


1
Erro ao gerar dict nf: page not in document
Erro ao gerar o json: name 'nro_nota' is not defined
i:  2

diretorio: Batch_17 nro: 2 | doc: 12___otica_do_cizinho.pdf | pdf?: True | pesquisavel?: True | paginas: 1

pdf_realmente_pequisavel:  (True, 0)

PDF Pesquisavel diretorio: Batch_17 nro: 2 | doc: 12___otica_do_cizinho.pdf | pdf?: True | pesquisavel?: True | paginas: 1

27/07/2023, 15:36
Nota Fiscal de Serviços Eletrônica (NFSe)
PREFEITURA MUNICIPAL DE MAGE
SECRETARIA MUNICIPAL DA FAZENDA
NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e
Número da Nota:
12
Competência:
Julho/2023
Data e Hora da Emissão:
27/07/20

## Desenha Box

In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/images/processadas/Doria Marinho 0297 Raquel.pdf.jpg"

In [ ]:
# To draw everything
trattempl. draw_box_model(model, boundaries_info, sections_info, frames_info, field_boxes_info)


In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


# Outros

In [ ]:
# nf_data_servico = {}#VERIFICAR
# analise_doc_nf = {} #VERIFICAR
# file_data = [] #VERIFICAR
list_document_pages = []
#nro_nota = 0
# # TEMP
# # Nome do arquivo json
# nome_formado_json = batch_name +".json"
#3. path formado para nome do arquivo json
# json_file_path = os.path.join(json_path, nome_formado_json)
#root_doc_analise = os.path.join(documentos_extracao_path, batch_name)
#print(root_doc_analise)

In [ ]:
import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron
import modules.trata_dicionarios as tratdic
import modules.trata_imagem as tratimg
import modules.trata_files as tratfiles
import modules.trata_texto_extraido as trattext
import modules.trata_template as trattemplate


In [ ]:
#### IMPORTANTE - NRO BATCH PARA TESTE    0 = PDF_PESQUISAVEL | 1 = RASTER_PDF

i_test = 1

tipo_pdf = []
tipo_pdf.append('PDF_PESQUISAVEL')
tipo_pdf.append('RASTER_PDF')
tipo_pdf[i_test]


# Tratamento do Path de ORIGEM DO DOCUMENTOS PARA TESTE QUE SERAO MOVIDOS
list_path_test = []
list_path_test.append("pipeline_extracao_documentos/4_area_testes/pdf_pesquisavel_4_test")
list_path_test.append("pipeline_extracao_documentos/4_area_testes/raster_pdf_4_test")
list_path_test[i_test]

# Frame para teste
i_frame = 0

frames_pesquisa = []
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == model]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_name = row_frame['label']
    frames_pesquisa.append(frame_name)

# Nome Batch
batch_name = "Batch_" + str(tipo_pdf[i_test]) + "_" + str(i_frame)

# Nome do arquivo json
nome_formado_json = batch_name +".json"

# Listagem dos frames de pesquisa
i = 0
for frame in frames_pesquisa:
    print(f'seq ={i:>3} | {frame}')
    i += 1
    
if frames_pesquisa[i_frame]:
    print(f'\n\nDados do teste: batch_name: {batch_name} | frame: {frames_pesquisa[i_frame]} | model: {model} | tipo_pdf: {tipo_pdf[i_test]}')
    
    
######### PATHS
#1. path formado para busca de pdfs recursiva
root_doc_analise = os.path.join(documentos_extracao_path, batch_name)

#2. path para documentos teste RASTER PDF (ATRIBIDO DA LISTA)
path_test_pdf = list_path_test[1]

#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)

#Listando paths utilizados
#print(f'\nroot_doc_analise: {root_doc_analise}\npath_test_pdf: {path_test_pdf}\njson_file_path: {json_file_path}')  

In [ ]:

import modules.trata_dicionarios as tratdic

import modules.trata_files as tratfiles
import modules.trata_texto_extraido as trattext
import modules.trata_template as trattemplate

In [ ]:
# Trata files
# 3. Ajusta o filename tirando caracteres especiais 
def conv_filename(title):
    
    # Divida o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    # Adicione a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 4. Ajusta o filename tirando caracteres especiais e a
def conv_filename_no_ext(title):
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename  


# move NF processadas ok
def move_raster_pdf(document_path, raster_pdf_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(raster_pdf_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message  
    
    
    # 5. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable_analise(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        pages = pdf_document.page_count
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf, pages
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False    